In [2]:
from communication_system import *
from deep_receiver import *
import pickle

retrain = False

# test1：AWGN channel
# generate training set 
train_set_sample_num =  2 * 1000 * 9
data_bits = gen_rand_code(32, train_set_sample_num)

moder = {
    "bpsk": PSKModem(2),
    "qpsk": PSKModem(4)
}


class DeepSet:
    def __init__(self, mod_name , noise):
        self.mod = moder[mod_name]
        self.mod_name = mod_name
        self.noise = noise
        self.net = Conv_DenseNet_1D()
        self.train_ebr_log = None
        self.test_ebr = None

var_file = "data\\additional_noise_deep_set.data"

deep_set = None

if not os.path.exists(var_file) or retrain:
    print("need to train nets")
    deep_set = [
        DeepSet("bpsk", 'awgn'),
        DeepSet("qpsk",'awgn'),
        DeepSet("bpsk", 'aggn'),
        DeepSet("qpsk", 'aggn')
    ]
    
    ham74 = Hamming74()
    coded_data_bits = ham74.encode(data_bits)
    EbN0 = np.arange(9)
    for setting in deep_set:
        mod_data, _, _ = modulate(coded_data_bits, setting.mod, True)
        
        # generate noise
        SNR = EbN02SNR(EbN0, setting.mod.num_bits_symbol, 8)
        
        # generate training set
        noised_data = []
        for mod_data_part, snr in zip(mod_data.reshape([9, len(mod_data)//9, -1]), SNR):
            noised_data.append(add_noise(mod_data_part, setting.noise, snr))
        noised_data = np.concatenate(noised_data, 0)
        train_set = (torch.tensor(noised_data), torch.tensor(data_bits))
        
        # 训练模型
        print(f"net(modulate:{setting.mod}, noise:{setting.noise}) start training")
        setting.train_ebr_log = train(
            net = setting.net,
            data_set = train_set,
            batch_size = 256,
            lr = 0.001,
            lr_gain = 0.1,
            epochs = 8,
            steady_epochs = 2,
            momentum = 0.9
        )
        print(f"net(modulate:{setting.mod}, noise:{setting.noise}) training done")
        
    # 存储模型变量
    print(f"save the data to the file:{var_file}")
    fp = open(var_file, "wb")
    pickle.dump(deep_set, fp)
    fp.close()
else:
    print(f"load the data from the file:{var_file}")
    fp = open(var_file, "rb")
    deep_set = pickle.load(fp)
    fp.close()
   


In [ ]:
Test_Model_V_fangan67_lite.cpp

commpy.modulation.PSKModem

In [33]:
# 
delta_f = np.random.uniform(-0.01, 0.01, 1000)
signal_f_off_data = [add_frequency_offset(signal, 8, 0.01) for signal in signal_data]

In [ ]:
# IQ imbalance
for i, j in zip(np.arange(12), np.repeat(np.arange(12).reshape([-1, 1]), 3, 1)):
    print(i, j)